### Image augmentation with `Augmentor`

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
%autosave 60

Autosaving every 60 seconds


In [6]:
import numpy as np

from fastai import *
from fastai.vision import *

In [4]:
import Augmentor

In [7]:
np.random.seed(1138)

In [5]:
mimage_path = "../data/images/Museum"

In [ ]:
# classes = ['Nar', 'Nmi', 'Nfr', 'Nhe']

Augment a batch of images

In [ ]:
p = Augmentor.Pipeline(mimage_path)

In [ ]:
p.rotate(probability = 1.0, max_left_rotation = 5, 
         max_right_rotation = 10)
p.zoom_random(probability = 0.5, percentage_area = 0.8)
p.flip_left_right(probability = 0.5)
p.flip_top_bottom(probability=0.3)
p.random_distortion(probability = 1, grid_width = 4, grid_height = 4, 
                    magnitude = 8)

In [ ]:
p.status()

Now sample from that pipeline:

In [ ]:
p.sample(1000)

In [6]:
aug_path = "../data/images/Museum/output"

In [7]:
data_64 = ImageDataBunch.from_folder(path = aug_path, train = ".", 
                                         valid_pct = 0.2, size = 64, num_workers = 4, 
                                         bs = 64).normalize(imagenet_stats)

In [8]:
data_64

ImageDataBunch;

Train: LabelList (800 items)
x: ImageList
Image (3, 64, 64),Image (3, 64, 64),Image (3, 64, 64),Image (3, 64, 64),Image (3, 64, 64)
y: CategoryList
Nar,Nar,Nar,Nar,Nar
Path: ../data/images/Museum/output;

Valid: LabelList (200 items)
x: ImageList
Image (3, 64, 64),Image (3, 64, 64),Image (3, 64, 64),Image (3, 64, 64),Image (3, 64, 64)
y: CategoryList
Nar,Nar,Nmi,Nmi,Nmi
Path: ../data/images/Museum/output;

Test: None

In [ ]:
print(data_64.classes)

In [ ]:
data_64.show_batch(rows = 3, figsize = (6, 6))

### Now the `ResNet34`

In [9]:
# with 64 sized images
learn_rn34 = cnn_learner(data_64, \
                            models.resnet34, \
                            metrics = error_rate)

In [ ]:
learn_rn34.lr_find()

In [ ]:
learn_rn34.recorder.plot()

In [10]:
learn_rn34.fit_one_cycle(5, max_lr = slice(1e-03, 1e-02)) 
# with specified LR, get error rate of ~15% after 5 cycles

epoch,train_loss,valid_loss,error_rate,time
0,1.479138,1.102509,0.240000,00:29
1,0.988070,0.857725,0.185000,00:27
2,0.712620,0.576305,0.180000,00:27
3,0.555420,0.543126,0.155000,00:27
4,0.413785,0.505454,0.155000,00:28
